<h1 style="color:blue;"> LANGUAGE TRANSLATION </h1>

## Translates from English to French

### Models
In this section, 4 different models have been used.

 - Model 1 is a simple LSTM
 - Model 2 is a LSTM with Embedding
 - Model 3 is a Bidirectional LSTM with Embedding
 - <b><h3 style="color:red;">Model 4 is an <U>Encoder-Decoder LSTM </U> with Bidirectional LSTM and Embedding </h3></b>

In [1]:
import os
import pickle
import copy
import numpy as np

In [2]:
import collections
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Input, Dense, TimeDistributed, Activation, RepeatVector
from tensorflow.keras.layers import Bidirectional, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.utils import plot_model

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7372667322577924378
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4963368960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10027432836746122393
physical_device_desc: "device: 0, name: GeForce RTX 2060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


### Load Data

In [4]:
def load_data(input_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        data = f.read()

    return data.split('\n')

In [5]:
english_sentences  = load_data('small_vocab_en.txt')
french_sentences  = load_data('small_vocab_fr.txt')

In [6]:
for sample_i in range(5):
    print('English sample {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('French sample {}:  {}\n'.format(sample_i + 1, french_sentences[sample_i]))

English sample 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
French sample 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .

English sample 2:  the united states is usually chilly during july , and it is usually freezing in november .
French sample 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .

English sample 3:  california is usually quiet during march , and it is usually hot in june .
French sample 3:  california est généralement calme en mars , et il est généralement chaud en juin .

English sample 4:  the united states is sometimes mild during june , and it is cold in september .
French sample 4:  les états-unis est parfois légère en juin , et il fait froid en septembre .

English sample 5:  your least liked fruit is the grape , but my least liked is the apple .
French sample 5:  votre moins aimé fruit est le raisin , mais mon moins aimé est la pomme .



### Vocabulary

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess

For this project, we will convert the text into sequences of integers using the following preprocess methods:

Tokenize the words into ids
Add padding to make all the sequences the same length.

### Tokenize 

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [9]:
# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}


In [10]:
text_tokenized

[[1, 2, 4, 5, 6, 7, 1, 8, 9],
 [10, 11, 12, 2, 13, 14, 15, 16, 3, 17],
 [18, 19, 3, 20, 21]]

In [11]:
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print(f'Sequence {sample_i + 1} in x')
    print(f'  Input:  {sent}')
    print(f'  Output: {token_sent}')

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding 

When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the end of each sequence

In [12]:
def pad(x, length=20):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')


# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print(f'Sequence {sample_i + 1} in x')
    print(f'  Input:  {token_sent}')
    print(f'  Output: {pad_sent}')

Sequence 1 in x
  Input:  [1, 2, 4, 5, 6, 7, 1, 8, 9]
  Output: [1 2 4 5 6 7 1 8 9 0 0 0 0 0 0 0 0 0 0 0]
Sequence 2 in x
  Input:  [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
  Output: [10 11 12  2 13 14 15 16  3 17  0  0  0  0  0  0  0  0  0  0]
Sequence 3 in x
  Input:  [18, 19, 3, 20, 21]
  Output: [18 19  3 20 21  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


### Preprocess Pipeline
 - Combines Tokenizing  & Padding

In [13]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk


In [14]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 20
Max French sentence length: 20
English vocabulary size: 199
French vocabulary size: 344


In [26]:
print(preproc_english_sentences.shape)
print(preproc_french_sentences.shape)

(137860, 20)
(137860, 20, 1)


## Models

In [16]:
# The neural network will be translating the input to words ids, which isn't the final form we want. 
#  We want the French translation. The function logits_to_text will bridge the gab between the logits from the neural network
#  to the French translation. We'll be using this function to better understand the output of the neural network.

def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    #index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words =  tokenizer.index_word
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


## MODEL 1

In [17]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [18]:

# Reshaping the input to work with a basic RNN
print(preproc_english_sentences.shape)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(tmp_x.shape)


(137860, 20)
(137860, 20)
(137860, 20, 1)


In [19]:

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

print(simple_rnn_model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 256)           264192    
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 1024)          263168    
_________________________________________________________________
dropout (Dropout)            (None, 20, 1024)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 344)           352600    
Total params: 879,960
Trainable params: 879,960
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
#plot_model(simple_rnn_model, to_file='model.png', show_shapes=True)

In [20]:
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 9s 79us/sample - loss: 2.0627 - accuracy: 0.5142 - val_loss: nan - val_accuracy: 0.6150
Epoch 2/10
110288/110288 [==============================] - 6s 52us/sample - loss: 1.2799 - accuracy: 0.6257 - val_loss: nan - val_accuracy: 0.6659
Epoch 3/10
110288/110288 [==============================] - 6s 53us/sample - loss: 1.1269 - accuracy: 0.6549 - val_loss: nan - val_accuracy: 0.6807
Epoch 4/10
110288/110288 [==============================] - 6s 53us/sample - loss: 1.0348 - accuracy: 0.6713 - val_loss: nan - val_accuracy: 0.6951
Epoch 5/10
110288/110288 [==============================] - 6s 54us/sample - loss: 0.9773 - accuracy: 0.6804 - val_loss: nan - val_accuracy: 0.6965
Epoch 6/10
110288/110288 [==============================] - 6s 53us/sample - loss: 0.9334 - accuracy: 0.6898 - val_loss: nan - val_accuracy: 0.7190
Epoch 7/10
110288/110288 [==============================] - 6

In [21]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois chaud en mois de il et il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


## MODEL 2

### - Embedding

In [23]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(LSTM(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


In [24]:
# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 256)           51200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 256)           525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 1024)          263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 1024)          0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 345)           353625    
Total params: 1,193,305
Trainable params: 1,193,305
Non-trainable params: 0
_________________________________________________________________


In [26]:
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 9s 83us/sample - loss: 1.6839 - accuracy: 0.6218 - val_loss: 0.5985 - val_accuracy: 0.8152
Epoch 2/10
110288/110288 [==============================] - 8s 70us/sample - loss: 0.4825 - accuracy: 0.8459 - val_loss: 0.3478 - val_accuracy: 0.8853
Epoch 3/10
110288/110288 [==============================] - 8s 68us/sample - loss: 0.3305 - accuracy: 0.8909 - val_loss: 0.2743 - val_accuracy: 0.9063
Epoch 4/10
110288/110288 [==============================] - 8s 68us/sample - loss: 0.2705 - accuracy: 0.9090 - val_loss: 0.2388 - val_accuracy: 0.9181
Epoch 5/10
110288/110288 [==============================] - 8s 69us/sample - loss: 0.2396 - accuracy: 0.9182 - val_loss: 0.2201 - val_accuracy: 0.9237
Epoch 6/10
110288/110288 [==============================] - 8s 69us/sample - loss: 0.2222 - accuracy: 0.9232 - val_loss: 0.2081 - val_accuracy: 0.9274
Epoch 7/10
110288/110288 [=================

In [27]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


## Model 3: Bidirectional RNNs

 - One restriction of a RNN is that it can't see the future input, only the past. This is where bidirectional recurrent neural networks come in. They are able to see the future data.

In [37]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [38]:
# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 256)           51200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 20, 256)           525312    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 20, 1024)          263168    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 1024)          0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 345)           353625    
Total params: 1,193,305
Trainable params: 1,193,305
Non-trainable params: 0
_________________________________________________________________


In [39]:
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 25s 231us/sample - loss: 1.6978 - accuracy: 0.6175 - val_loss: 0.6224 - val_accuracy: 0.8106
Epoch 2/10
110288/110288 [==============================] - 24s 218us/sample - loss: 0.4984 - accuracy: 0.8419 - val_loss: 0.3534 - val_accuracy: 0.8828
Epoch 3/10
110288/110288 [==============================] - 25s 224us/sample - loss: 0.3411 - accuracy: 0.8867 - val_loss: 0.2869 - val_accuracy: 0.9038
Epoch 4/10
110288/110288 [==============================] - 25s 223us/sample - loss: 0.2768 - accuracy: 0.9066 - val_loss: 0.2378 - val_accuracy: 0.9187
Epoch 5/10
110288/110288 [==============================] - 25s 224us/sample - loss: 0.2436 - accuracy: 0.9167 - val_loss: 0.2197 - val_accuracy: 0.9238
Epoch 6/10
110288/110288 [==============================] - 24s 219us/sample - loss: 0.2239 - accuracy: 0.9225 - val_loss: 0.2107 - val_accuracy: 0.9268
Epoch 7/10
110288/110288 [=====

In [40]:
# Print prediction(s)
print("Prediction:")
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

print("\nCorrect Translation:")
print(french_sentences[:1])

print("\nOriginal text:")
print(english_sentences[:1])

Prediction:
new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>

Correct Translation:
["new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."]

Original text:
['new jersey is sometimes quiet during autumn , and it is snowy in april .']


<h1 style="color:red;"> Model 4: Encoder-Decoder </h1>

In [24]:
def  model_final (input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.003

    # Build the layers    
    model = Sequential()
    # Embedding
    model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
                         input_shape=input_shape[1:]))
    # Encoder
    model.add(Bidirectional(LSTM(128, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dropout(0.2))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [25]:
    # TODO: Train neural network using model_final
model = model_final(preproc_english_sentences.shape,
                    preproc_french_sentences.shape[1],
                    len(english_tokenizer.word_index)+1,
                    len(english_tokenizer.word_index)+1)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 128)           25600     
_________________________________________________________________
bidirectional (Bidirectional (None, 20, 256)           263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 20, 256)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20, 256)          

In [28]:
model.fit(preproc_english_sentences, preproc_french_sentences, batch_size=512, epochs=25, validation_split=0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/25
110288/110288 [==============================] - 18s 164us/sample - loss: nan - accuracy: 0.3769 - val_loss: nan - val_accuracy: 0.3797
Epoch 2/25
110288/110288 [==============================] - 10s 93us/sample - loss: nan - accuracy: 0.3786 - val_loss: nan - val_accuracy: 0.3797
Epoch 3/25
110288/110288 [==============================] - 10s 93us/sample - loss: nan - accuracy: 0.3786 - val_loss: nan - val_accuracy: 0.3797
Epoch 4/25
110288/110288 [==============================] - 10s 94us/sample - loss: nan - accuracy: 0.3786 - val_loss: nan - val_accuracy: 0.3797
Epoch 5/25
110288/110288 [==============================] - 10s 94us/sample - loss: nan - accuracy: 0.3786 - val_loss: nan - val_accuracy: 0.3797
Epoch 6/25
110288/110288 [==============================] - 10s 93us/sample - loss: nan - accuracy: 0.3786 - val_loss: nan - val_accuracy: 0.3797
Epoch 7/25
110288/110288 [==============================] - 10s 93us/sam